In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from copy import copy, deepcopy
from scipy import special
import pandas as pd
import matplotlib as mpl
from matplotlib.colors import colorConverter
import time
import pystencils
import sympy
from lbmpy.session import *

In [ ]:
# SOR iterative method with objects

def SOR_with_objects(N, omega, object_array, conc_mat):
    """
        Function to find final diffusion gradient with objects.
        
        Parameters:
          - N: grid size (NxN)
          - omega : SOR equation constant
          - object_array: a numpy array of 0's and 1's that determine where
            the object is
          - conc_mat: a numpy array containing all concentrations per coordinate
        
        Returns:
        The updated concentration matrix for the diffusion gradient
    """
    
    # initialise parameters
    epsilon = 10 ** -5 # convergence threshold
    diff = 100000 # set to large value    

    # loop until convergence
    while diff > epsilon:
        conc_mat_old = deepcopy(conc_mat)
        conc_mat = SOR(conc_mat, omega, object_array)
        diff = np.amax(np.abs(conc_mat - conc_mat_old))
    return conc_mat

def SOR(conc_mat, omega, object_array):
    """
        Function to find diffusion gradient with objects.
        
        Parameters:
          - conc_mat: a numpy array containing all concentrations per coordinate
          - omega : SOR equation constant
          - object_array: a numpy array of 0's and 1's that determine where
            the object is

        Returns:
        The updated concentration matrix for the diffusion gradient
    """

    # Set inflow boundary to max concentration
    # TODO: this is sun diffusion specific so is liable to change
    conc_mat[N - 1] = 1
    
    # go through left and right boundaries 
    for k in range(1, N - 1):
        # left boundary 
        # if it is part of object, concentration is zero
        if object_array[k][0] == 1:
                conc_mat[k][0] = 0
        # otherwise: SOR equation
        else:
            conc_mat[k][0] = omega / 4 * (conc_mat[k + 1][0] + conc_mat[k - 1][0] + conc_mat[k][1] + conc_mat[k][N - 1]) + (1 - omega) * conc_mat[k][0]

        #right boundary
        # if it is part of object, concentration is zero
        if object_array[k][N - 1] == 1:
            conc_mat[k][N - 1] = 0
        # otherwise: SOR equation
        else:
            conc_mat[k][N - 1] = omega / 4 * (conc_mat[k + 1][N - 1] + conc_mat[k - 1][N - 1] + conc_mat[k][0] + conc_mat[k][N - 2]) + (1 - omega) * conc_mat[k][N - 1]

    # loop trough rest/middle of matrix
    for j in range(N - 1):
        for i in range(1, N - 1):
            # if part of object, concentration is zero
            if object_array[i][j] == 1:
                conc_mat[i][j] = 0
            # otherwise: SOR equation
            else:
                conc_mat[i][j] = (omega / 4) * (conc_mat[i + 1][j] + conc_mat[i - 1][j] + conc_mat[i][j + 1] + conc_mat[i][j - 1]) + (1 - omega) * conc_mat[i][j]
                if conc_mat[i][j] < 0:
                    conc_mat[i][j] = 0
  
    return conc_mat

In [ ]:
def get_candidates_SOR(object_loc, object_array, candidates):
    """
        Function to find the neighbours of an object cell,
        if they qualify as growth candidates, add to set 
        (so all candidates are unique) 
        
        Parameters:
          - object_loc: coordinates tuple of new object cell
          - object_array: a numpy array of 0's and 1's that determine where
            the object is
          - candidates: a set of tuples containing the candidate coordinates
        
        Returns:
        The updated candidate set
    """
    # Get object coordinates
    y = object_loc[0]
    x = object_loc[1]

    # check if edirect neighbours are NOT part of object, and add them to candidates
    if x != N - 1 and object_array[y][x + 1] == 0:
        candidates.add((y, x + 1))
        
    if x != 0 and object_array[y][x - 1] == 0:
        candidates.add((y, x - 1))
        
    if y != N - 1 and object_array[y + 1][x] == 0:
        candidates.add((y + 1, x))
        
    if y != 0 and object_array[y - 1][x] == 0:
        candidates.add((y - 1, x))

    return candidates


def SOR_DLA_to_solution(N, eta, omega, iterations):
    """
        Function to calculate the SOR of a grid with object, 
        until convergence, with growing object
        
        Parameters:
          - N: desired grid size (NxN)
          - eta: weight of concentration gradient for growth
          - omega: SOR equation constant
          - iterations: how many times the object should grow
        
        Returns:
        - The concentration matrix
        - The object_array matrix with fully grown object
    """
    # calculate the analytic solution at t = 1
    t = 1
    analytic_sol = lambda x, D, t: sum([scipy.special.erfc((1 - x + 2 * i) / (2 * np.sqrt(D * t))) - scipy.special.erfc((1 + x + 2 * i) / (2 * np.sqrt(D * t))) for i in range(10000)])
    x = np.arange(0, 1, 1 / N)
    analytic_matrix = analytic_sol(x, 1, t)
    seed_coor_x = int(N / 2)  # set seed at bottom middle of grid

    # initialise start diffusion gradient as the analytic solution
    conc_mat = np.zeros((N, N))
    for i in range(N):
        conc_mat[i] = analytic_matrix[i]

    # TODO: this is also done in SOR() <- SOR_with_objects()
    # and thus unnecessary here, but left just in case of 
    # nutrient diffusion addition making it needed here
    # (check after big merge)
    # source top boundary
    conc_mat[N - 1] = 1

    # initalisation of array with seed of object
    object_array = np.zeros((N, N))
    object_array[0, seed_coor_x] = 1
    candidates = set()
    candidates = get_candidates_SOR((0, seed_coor_x), object_array, candidates)
    
    # first SOR to initialize starting concentrations
    conc_mat = SOR_with_objects(N, omega, object_array, conc_mat)
     
    # loop until object is grown 'iterations' times, finding SOR with each growth
    for _ in range(iterations):
        object_array, candidates = choose_growth(N, eta, conc_mat, candidates, object_array)
        conc_mat = SOR_with_objects(N, omega, object_array, conc_mat)

    plt.imshow(conc_mat, origin='lower', extent=[0, 1, 0, 1], cmap='Spectral')
    
    return conc_mat, object_array


def choose_growth(N, eta, conc_mat, candidates, object_array):
    """
        Function to calculate growth probabilities of each candidate cell,
        choose one and grow it this timestep
        
        Parameters:
          - N: grid size (NxN)
          - eta: weight of concentration gradient for growth
          - conc_mat: numpy array containing all concentrations per coordinate
          - candidates: set of tuple coordinates of all growable cells 
        
        Returns:
        - Updated object_array matrix with newly grown object cell
        - Updated candidate set
    """
    
    # TODO: add necessary input constants as arguments (nutrient diff)
    
    probs = []  # store all candidate grow probabilities
    list_candidates = list(candidates)  # ensure same ordering 
    
    for i in candidates:
        prob_sun = (conc_mat[i] ** eta) / np.sum([conc_mat[cand] ** eta for cand in candidates])
        # TODO: calculate nutrient probability and combine
        # prob_nut =
        # probs.append(alpha * prob_sun + (1 - alpha) * prob_nut)
        probs.append(prob_sun)

    # choose a candidate and grow
    chosen_growth = list_candidates[np.random.choice(len(candidates), p=probs)]
    object_array[chosen_growth] = 1
    conc_mat[chosen_growth] = 0  # set conc to 0 where object has grown
    # update candidate set after growth
    candidates = get_candidates_SOR(chosen_growth, object_array, candidates)
    # delete newly grown cell from growth candidates
    candidates.remove(chosen_growth)
    
    return object_array, candidates
    

In [ ]:
# function to make a combined imshow plot, where the object is visible along with the gradient

## code (with small adjustments) based on answer at: https://stackoverflow.com/questions/10127284/overlay-imshow-plots-in-matplotlib
def plot_object_gradient(conc_mat, object_array, eta):
    # generate the colors for your colormap
    color1 = colorConverter.to_rgba('white')
    color2 = colorConverter.to_rgba('black')

    # make the colormaps
    cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap2', [color1,color2], 256)
    cmap2._init() # create the _lut array, with rgba values

    # create your alpha array and fill the colormap with them.
    # here it is progressive, but you can create whathever you want
    alphas = np.linspace(0, 0.8, cmap2.N + 3)
    cmap2._lut[:, -1] = alphas

    img2 = plt.imshow(conc_mat, interpolation='nearest', cmap='Spectral', origin='lower', extent=[0, 1, 0, 1])
    plt.colorbar()
    img3 = plt.imshow(object_array, interpolation='nearest', cmap=cmap2, origin='lower', extent=[0, 1, 0, 1])

    plt.title(f"Object with gradient, eta = {eta}")

    plt.show()

In [ ]:
# test run

# initialise parameters
N = 25
eta = 1 
omega = 1.5
iterations = 20

conc_mat, object_array = SOR_DLA_to_solution(N, eta,omega, iterations)
plot_object_gradient(conc_mat, object_array, 1)

In [ ]:
import pystencils
import sympy
from lbmpy.session import *
# example lid driven cavity
ldc = create_lid_driven_cavity(domain_size=(100, 40), method='srt', relaxation_rate=1.6)

ldc.run(500)
plt.figure(dpi=200)
#plt.vector_field(ldc.velocity_slice());

plt.scalar_field(ldc.velocity[:, :, 0])

## Convection-Diffusion Equation (nutrient concentration) 

In [ ]:
def diffusion_update(conc_mat,omega):
    """
    Function to calculate the difference in the nutrient concentration based on the diffusion.
    Uses SOR method, only right boundary is calculated (rest are sink boundaries, with standard set to 0)    
    Takes nutrient concentration matrix and omega (relaxation parameter) as input. 
    Outputs a matrix with the difference in nutrient concentration caused by diffusion. 
    """
    diff_update = np.zeros((N,N))

    # middle of matrix
    for j in range(N-1):
        for i in range(1,N-1):
            diff_update[i][j] = (omega/4)*(conc_mat[i+1][j] + conc_mat[i-1][j] + conc_mat[i][j+1] + conc_mat[i][j-1]) - omega*conc_mat[i][j]

    # right boundary
    for k in range(1,N-1):
        diff_update[k][N-1] = omega/4*(conc_mat[k+1][N-1] + conc_mat[k-1][N-1]  + conc_mat[k][N-2]) - omega*conc_mat[k][N-1] # + conc_mat[k][0]

    return diff_update

def convection_update(nutrients_matrix, scenario):
    
    """
    Calculates the difference in nutrient concentration caused by the convection (flow).
    Input: nutrient concentration matrix, 'scenario' = channel with lattice Boltzmann flow, velocity profile.
    Output: matrix with the difference in nutrient concentration caused by convection. 
    
    """
    conv_update = np.zeros((N,N))
    
    # flow to the right
    conv_flow_right = np.transpose(scenario.velocity_slice()[:-1,:,0])*nutrients_matrix[:,:-1]
    conv_flow_right[np.where(conv_flow_right < 0)] = 0
    conv_update[:,:-1] -= conv_flow_right
    conv_update[:,1:] += conv_flow_right
    
    # flow to the left
    conv_flow_left = np.transpose(scenario.velocity_slice()[2:,:,0])*nutrients_matrix[:,2:]
    conv_flow_left[np.where(conv_flow_left > 0)] = 0
    conv_update[:,2:] -= conv_flow_left
    conv_update[:,1:-1] += conv_flow_left
    
    # flow up 
    conv_flow_up = np.transpose(scenario.velocity_slice()[1:,:-1,1])*nutrients_matrix[:-1,1:]
    conv_flow_up[np.where(conv_flow_up < 0)] = 0
    conv_update[:-1,1:] -= conv_flow_up
    conv_update[1:,1:] += conv_flow_up
    
    #flow down
    conv_flow_down = np.transpose(scenario.velocity_slice()[1:,1:,1])*nutrients_matrix[1:,1:]
    conv_flow_down[np.where(conv_flow_down > 0)] = 0
    conv_update[1:,1:] -= conv_flow_down
    conv_update[:-1,1:] += conv_flow_down
    
    return conv_update

In [ ]:
N = 100

# function to be able to activate and deactivate inflow speed
def velocity_info_callback(boundary_data, activate=True, **_):
    boundary_data['vel_1'] = 0
    if activate==True:
        u_max = 0.05
        boundary_data['vel_0'] = u_max 
    else:
        boundary_data['vel_0'] = 0
        
# function to make a figure of the coral with the flow 
def figure_coral_with_flow(scenario):
    plt.figure(dpi=200)
    plt.scalar_field(scenario.velocity[:,:,0])
    #plt.vector_field(scenario.velocity_slice());
    plt.colorbar()
    plt.show()
    

# function to grow the object (by one) and to update the flow (lbm, ten steps)    
def grow_and_flow(object_array, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, i):
    # set noslip boundary around object, update flow
    object_array,conc_mat, candidates = choose_growth(N,eta,nutrients_matrix, candidates)
    for j in range(N-1):
        for i in range(1,N-1):
            if object_array[i][j] == 1:
                flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
    
    # run lbm 10 steps, saving at each step for movie
    for _ in range(10):
        scenario.run(1)
        snappy = deepcopy(scenario.velocity[:,:,0])
        snapshots_vel.append(snappy)
    return object_array, candidates, scenario, snapshots_vel

# initiate and advance a channel with lattice Boltzmann
scenario = create_channel(domain_size=(N,N), force=1e-4,duct=True, method='srt', relaxation_rate=1.9)

# in- and outflow boundary
stencil = get_stencil("D2Q9")
outflow = ExtrapolationOutflow(stencil[4], scenario.method)
scenario.boundary_handling.set_boundary(outflow, make_slice[:,N])
inflow = UBB(velocity_info_callback, dim=scenario.method.dim)
scenario.boundary_handling.set_boundary(inflow, make_slice[0, :])

# keep track whether flow is on or of
activ = True

# list to save the snapshots
snapshots_vel = []

# 50 lbm steps, saving a snapshot of each flow update
for _ in range(50):
    scenario.run(1)
    snappy = deepcopy(scenario.velocity[:,:,0])
    snapshots_vel.append(snappy)

# initalisation of array with seed of object
object_array = np.zeros((N,N))
object_array[0][int(N/2)] = 1
candidates = set()
candidates = get_candidates_SOR((0,int(N/2)), object_array, candidates)

# initiation of the nutrient matrix
nutrients_matrix = np.zeros((N,N))
nutrients_matrix[:,0:5] = 1
omega = 0.7
eta = 1

# diffusion + convection for x timesteps
snapshots_grad = []
for iteration in range(2000):
    # every 200 iterations, grow object, set noslip boundary and run LBM
    if iteration%50==49:
        if activ==True:
            scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=False)
            activ = False
        elif activ==False:
            scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=True)
            activ = True
            
        for _ in range(5):
            object_array, candidates, scenario, snapshots_vel = grow_and_flow(object_array, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, iteration)   

        figure_coral_with_flow(scenario)
        plot_object_gradient(nutrients_matrix, object_array, 1)

    # every step, update the nutrient concentration matrix
    diffu_update = diffusion_update(nutrients_matrix,omega)
    conve_update = convection_update(nutrients_matrix, scenario)
    nutrients_matrix = nutrients_matrix + conve_update + diffu_update
    # check if nutrient concentration does not become negative
    for i in range(N):
        for j in range(N):
            if nutrients_matrix[i][j] < 0:
                 nutrients_matrix[i][j] = 0
            if math.isnan(nutrients_matrix[i][j]):
                print(nutrients_matrix)
            # concentration is 0 at object
            if object_array[i][j] == 1:
                nutrients_matrix[i][j] = 0
    nutrients_matrix[:,0:int(N/10)] = 0.3
    nutrients_matrix[:,-1] = 0
    nutrients_matrix[0,:] = 0
    nutrients_matrix[-1,:] = 0
    snapshots_grad.append(nutrients_matrix)
    
plot_object_gradient(nutrients_matrix, object_array, 1)

In [ ]:
######### FILMPJEEEE

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(dpi=200 ,figsize=(8,8))
plt.title('Velocity field with growing object')

a = snapshots_vel[0]
im = plt.scalar_field(np.transpose(a))
cb = fig.colorbar(im)

#bar = plt.colorbar(hoi)
totaltime = 2000
def animate_func(i):
    vmax = np.max(snapshots_vel[i])
    vmin = np.min(snapshots_vel[i])
    im.set_data(np.transpose(snapshots_vel[i]))
    im.set_clim(vmin, vmax)
    
    #im.set_array(snapshots_vel[i])
    return [im]

anim = animation.FuncAnimation(fig,animate_func, frames = totaltime-1, interval=10, blit=True)
anim.save('velocity_coralgrowth_with_waves.mp4')

In [ ]:
import math

# The threshold which the pressure needs to reach before the coral breaks (disappears)
THRESHOLD = 0.005

def coral_breaky_breaky(seed_coord_x, coral_matrix, vector_field, copy=True):
    """
        Function that computes if the coral is gonna breaky breaky
        
        Parameters:
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - coral_matrix: a numpy array of 0's and 1's that determine where
            the coral is
          - vector_field: (I assume?) een numpy array of 2D vectors
          - copy: If True, does not modify the original object but instead returns a new one
          
        Returns:
          - The coral matrix with the relevant pixels removed
          - Broken boolean: True if part of coral got removed, else False
          
        O.O does it work?
    """
    
    # Copy the matrix if the user so desires
    if copy:
        coral_matrix = coral_matrix.copy()
    
    # Keep track whether part of coral has broken off
    broken = False
        
    # Loop through the coral matrix to find the corals
    for y in range(len(coral_matrix)):
        for x in range(len(coral_matrix[y])):
            if (x == seed_coord_x and y == 0) or coral_matrix[y][x] == 0: continue

            # Compute the pressure at this point (i.e., length of the vector)
            pressure = math.sqrt(vector_field[x, y][0] ** 2 + vector_field[x, y][1] ** 2)

            # If the pressure exceeds the threshold, remove the coral (:()
            if pressure > THRESHOLD:
                coral_matrix[y][x] = 0
                broken = True

    # We're done! Return the results
    return coral_matrix, broken

In [ ]:
def coral_painty_painty(seed_coord_x, coral_matrix, copy=True):
    """
        Function that checks which pixels are still connected to the source,
        and removes them. Also returns a new list of potential growth
        candidates.
        
        Note: We assume that a diagonal connection == no connection
        
        Parameters:
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - coral_matrix: a numpy array of 0's and 1's that determine where
            the coral is
          - copy: If True, does not modify the original object but instead returns a new one
        
        Returns:
        A tuple of:
          - The coral matrix, with all the unconnected pixels removed
          - A new list of growth candidates
    """
    
    # Copy the matrix if the user so desires
    if copy:
        coral_matrix = coral_matrix.copy()
        
    # Do a breadth-first search starting at the seed to see which pixels are connected to the seed
    coral_matrix[0][seed_coord_x] = 2
    to_do = [(seed_coord_x, 0)]
    candidates = set()
    while len(to_do) > 0:
        # Fetch the pixel to check
        x, y = to_do[0]
        to_do = to_do[1:]
        
        # Get the area around the pixel
        for neighbour in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            nx = x + neighbour[0]
            ny = y + neighbour[1]
            
            # Skip if the pixel is out-of-bounds
            if nx < 0 or nx > coral_matrix.shape[0] - 1 or ny < 0 or ny > coral_matrix.shape[1] - 1:
                continue
            
            # If the pixel is not a pixel, then store it as possible growth candidate
            if coral_matrix[ny][nx] == 0:
                # Uncomment for the correct candidates order
#                 candidates.add((nx, ny))
                candidates.add((ny, nx))
            
            # If it is an (unvisited) pixel, then mark as visited/connected and add it to the todo list
            if coral_matrix[ny][nx] == 1:
                # Mark the pixel as connected
                coral_matrix[ny][nx] = 2
                
                # Add to the queue
                to_do.append((nx, ny))
    
    # Go thru the matrix again and remove anything that's a 1
    coral_matrix[coral_matrix == 1] = 0
    # Convert the visited pixels back to 1's
    coral_matrix[coral_matrix == 2] = 1
    
    # Done!
    return coral_matrix, candidates


In [ ]:
def coral_density(seed_coord_x, coral_matrix):
    """
        Function that computes the average distance per pixel to the source
        for the entire coral.
        
        Parameters:
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - coral_matrix: a numpy array of 0's and 1's that determine where
            the coral is
        
        Returns:
        The average distance of the coral. The lower, the denser.
    """
    
    # Search through the coral
    total_distance = 0
    n_pixels = 0
    for y in range(len(coral_matrix)):
        for x in range(len(coral_matrix[y])):
            # If not a coral or the source block, then skip
            if (x == seed_coord_x and y == 0) or coral_matrix[y][x] != 1: continue

            # If coral, then compute the distance to the source block
            total_distance += math.sqrt((x - seed_coord_x)**2 + y**2)
            n_pixels += 1

    # To return the average distance, we return total / count
    return total_distance / n_pixels

In [ ]:
print(object_array[object_array>=1].shape)

In [ ]:
object_array_test = object_array.copy()

test_space = int(N / 4)
object_array_test[test_space, test_space] = 1
object_array_test[test_space + 1, test_space] = 1
object_array_test[test_space, test_space + 1] = 1
object_array_test[test_space, test_space] = 1

object_array_trimmed, candidates_trimmed = coral_painty_painty(int(N / 2), object_array_test, True)

In [ ]:
scenario = create_channel(domain_size=(100,100), force=1e-4,duct=True, method='srt', relaxation_rate=1.9)
#flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[0.3:0.4, 0.0:0.3])
#print(np.where(object_array==1))


for j in range(N-1):
    for i in range(1,N-1):
        # if part of object, concentration is zero
            if object_array_test[i][j] == 1:
                flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
scenario.run(500)

plt.figure()
plt.scalar_field(scenario.velocity[:,:,0])
# plt.vector_field(scenario.velocity_slice());

scenario = create_channel(domain_size=(100,100), force=1e-4,duct=True, method='srt', relaxation_rate=1.9)
#flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[0.3:0.4, 0.0:0.3])
#print(np.where(object_array==1))


for j in range(N-1):
    for i in range(1,N-1):
        # if part of object, concentration is zero
            if object_array_trimmed[i][j] == 1:
                flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
scenario.run(500)

plt.figure()
plt.scalar_field(scenario.velocity[:,:,0])
# plt.vector_field(scenario.velocity_slice());

scenario = create_channel(domain_size=(100,100), force=1e-4,duct=True, method='srt', relaxation_rate=1.9)
#flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[0.3:0.4, 0.0:0.3])
#print(np.where(object_array==1))


for i, j in candidates_trimmed:
    flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
scenario.run(500)

plt.figure()
plt.scalar_field(scenario.velocity[:,:,0])
# plt.vector_field(scenario.velocity_slice());

In [ ]:
print(coral_density(50, object_array_trimmed))